# 0. Load Data

In [1]:
import math 
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from joblib import dump

# Isolation Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# VAE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import metrics

In [2]:
df = pd.read_csv("data/cleaned_KPIs.csv")
df.head()

,index,cell_id,DL_TRAFFIC_VOLUME,UL_TRAFFIC_VOLUME,Inter_X2_based_HO_prep,VoLTE_total_traffic,INTRA_FREQ_HO_SR_RATIO,RRC_SR_RATIO,CELL_AVAILABILITY_RATIO,RACH_Stp_Completion_SR_RATIO,...,DCR_LTE_RATIO,CSSR_LTE_RATIO,LTE_INTER_ENODEB_HOSR_RATIO,E_UTRAN_Inter_Freq_HO_SR_RATIO,Inter_RAT_HO_SR_GERAN_SRVCC_RATIO,Inter_RAT_Total_HO_SR_RATIO,E_UTRAN_tot_HO_SR_inter_eNB_X2_RATIO,E_RAB_DR_RATIO,HOUR,cell
0,2021-05-09 00:00:00,2.226537e+17,3.779737e+10,3.947172e+09,15.0,4727.0,0.809859,0.992427,1.0,0.962688,...,0.001761,0.996041,0.400000,0.770642,0.963636,0.963636,0.400000,0.001761,0,0
1,2021-05-09 01:00:00,2.226537e+17,3.684898e+10,4.088752e+09,6.0,3076.0,0.886792,0.993288,1.0,0.973207,...,0.002468,0.995465,0.500000,0.842105,1.000000,1.000000,0.500000,0.002468,1,0
2,2021-05-09 02:00:00,2.226537e+17,3.292677e+10,5.016897e+09,8.0,3501.0,0.938356,0.994664,1.0,0.966330,...,0.003077,0.996044,0.375000,0.931624,1.000000,0.966667,0.375000,0.003077,2,0
3,2021-05-09 03:00:00,2.226537e+17,3.021547e+10,5.139107e+09,9.0,2275.0,0.860215,0.994819,1.0,0.943216,...,0.001721,0.995920,0.777778,0.816901,0.947368,0.947368,0.777778,0.001721,3,0
4,2021-05-09 04:00:00,2.226537e+17,3.082176e+10,4.250716e+09,17.0,2178.0,0.840426,0.995952,1.0,0.936256,...,0.002213,0.995628,0.764706,0.794521,1.000000,1.000000,0.764706,0.002213,4,0


In [3]:
df = df.drop(['cell_id','index'],axis=1)

# 1. Isolation Forest

In [4]:
IF=IsolationForest(n_estimators=150, 
                      max_samples ='auto', 
                      max_features=1,random_state=42)

In [5]:
IF.fit(df)
# score_samples = - score  
IF_score = -1 * IF.score_samples(df)
IF_score[:20]

array([0.51295825, 0.48509757, 0.53648751, 0.50925568, 0.48977436,
       0.52157804, 0.50460632, 0.47403398, 0.47869537, 0.4786467 ,
       0.46769137, 0.46808774, 0.47676634, 0.48726571, 0.48067503,
       0.50214502, 0.47844321, 0.47284323, 0.46897991, 0.46007395])

# 2. VAE

[keras - Variational AutoEncoder 官方代码范例(复杂)](https://keras.io/examples/generative/vae/)

[keras - AE/VAE 相对简单的代码构建](https://blog.keras.io/building-autoencoders-in-keras.html)

[keras - 中等程度AVE 但是有点乱](https://github.com/keras-team/keras/blob/2c8d1d03599cc03243bce8f07ed9c4a3d5f384f9/examples/variational_autoencoder.py)







[中文VAE理论+代码](https://blog.csdn.net/weixin_37737254/article/details/102920263)

[keras - 函数式API](https://keras.io/guides/functional_api/)

[keras - Conv1D](https://keras-zh.readthedocs.io/layers/convolutional/)


## 2.1 Build model

**(1) Standardize & Expand dimension**

In [9]:
df_scaled = StandardScaler().fit_transform(df)
df_expand = np.expand_dims(df_scaled,axis=1)
df_expand.shape

(8279, 1, 20)

**(2) Define sampling function**

In [10]:
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

**(3) Build Encoder**

In [24]:
original_dim = df.shape[1]
intermediate_dim = 32
latent_dim = 2

# encoder input
inputs = keras.Input(shape=(1,original_dim))
x = layers.Conv1D(filters=16, kernel_size=3, 
                  padding="same", activation="relu")(inputs)
x = layers.Conv1D(filters=4, kernel_size=3, 
                  padding="same", activation="relu")(x)
x = layers.Flatten()(x)

# the output of encoder
z_mean      = layers.Dense(latent_dim, name="z_mean")(x)
z_log_sigma = layers.Dense(latent_dim, name="z_log_var")(x)
z           = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1, 20)]      0           []                               
                                                                                                  
 conv1d_4 (Conv1D)              (None, 1, 16)        976         ['input_3[0][0]']                
                                                                                                  
 conv1d_5 (Conv1D)              (None, 1, 4)         196         ['conv1d_4[0][0]']               
                                                                                                  
 flatten_2 (Flatten)            (None, 4)            0           ['conv1d_5[0][0]']               
                                                                                            

**(4) Build Decoder**

In [25]:
# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(4, activation='relu')(latent_inputs)
x = layers.Reshape((1,4))(x)
x = layers.Conv1DTranspose(filters=16, kernel_size=3,
                                 padding="same", activation='relu')(x)
outputs = layers.Conv1DTranspose(filters=original_dim, kernel_size=3,
                                 padding="same", activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 2)]               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 12        
                                                                 
 reshape_2 (Reshape)         (None, 1, 4)              0         
                                                                 
 conv1d_transpose_4 (Conv1DT  (None, 1, 16)            208       
 ranspose)                                                       
                                                                 
 conv1d_transpose_5 (Conv1DT  (None, 1, 20)            980       
 ranspose)                                                       
                                                                 
Total params: 1,200
Trainable params: 1,200
Non-trainable p

**(5) Build Variational AutoEncoder**

In [26]:
# Instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

# Define Loss Function
reconstruction_loss = keras.losses.mean_squared_error(inputs, outputs)
kl_loss = -0.5 * K.sum(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
vae_loss = K.mean(reconstruction_loss + 0.01* kl_loss) 

# Optimiser 
adam = keras.optimizers.Adam(learning_rate=0.001)
# Add loss function
vae.add_loss(vae_loss)
vae.compile(optimizer=adam)


Notices:
- weight of kl_loss?  [theoretical insight](https://stats.stackexchange.com/questions/332179/how-to-weight-kld-loss-vs-reconstruction-loss-in-variational-auto-encoder) + [a more specific example](https://stats.stackexchange.com/questions/341954/balancing-reconstruction-vs-kl-loss-variational-autoencoder)
- reconsctruction_loss : if use `binary_crossentropy`( $BCE(y,p)=-y.log(p)-(1-y).log(1-p)$ ), the input value should be in 0~1. In our case, since we use 'StandardScaler', we'd better use `mse`, otherwise, when training the model, the loss becomes -inf 

**(6) Train model**

In [27]:
epochs = 500
batch_size = 128
callback = EarlyStopping(monitor='val_loss',patience=10)

# Train model
vae.fit(df_expand, df_expand,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(df_expand, df_expand),
        shuffle=True,
#         verbose=False,
        callbacks=[callback]
       )

Epoch 1/500
65/65 [==============================] - 1s 9ms/step - loss: 1.2305 - val_loss: 1.1909
Epoch 2/500
65/65 [==============================] - 0s 5ms/step - loss: 1.0999 - val_loss: 1.0308
Epoch 3/500
65/65 [==============================] - 0s 4ms/step - loss: 1.0095 - val_loss: 0.9940
Epoch 4/500
65/65 [==============================] - 0s 4ms/step - loss: 0.9804 - val_loss: 0.9612
Epoch 5/500
65/65 [==============================] - 0s 4ms/step - loss: 0.9449 - val_loss: 0.9264
Epoch 6/500
65/65 [==============================] - 0s 4ms/step - loss: 0.9152 - val_loss: 0.9020
Epoch 7/500
65/65 [==============================] - 0s 4ms/step - loss: 0.8943 - val_loss: 0.8829
Epoch 8/500
65/65 [==============================] - 0s 4ms/step - loss: 0.8752 - val_loss: 0.8628
Epoch 9/500
65/65 [==============================] - 0s 4ms/step - loss: 0.8552 - val_loss: 0.8432
Epoch 10/500
65/65 [==============================] - 0s 4ms/step - loss: 0.8369 - val_loss: 0.8307
Epoch 11/

65/65 [==============================] - 0s 4ms/step - loss: 0.7668 - val_loss: 0.7650
Epoch 84/500
65/65 [==============================] - 0s 5ms/step - loss: 0.7655 - val_loss: 0.7647
Epoch 85/500
65/65 [==============================] - 0s 4ms/step - loss: 0.7660 - val_loss: 0.7650
Epoch 86/500
65/65 [==============================] - 0s 4ms/step - loss: 0.7655 - val_loss: 0.7644
Epoch 87/500
65/65 [==============================] - 0s 4ms/step - loss: 0.7648 - val_loss: 0.7646
Epoch 88/500
65/65 [==============================] - 0s 4ms/step - loss: 0.7652 - val_loss: 0.7644
Epoch 89/500
65/65 [==============================] - 0s 4ms/step - loss: 0.7648 - val_loss: 0.7644
Epoch 90/500
65/65 [==============================] - 0s 4ms/step - loss: 0.7670 - val_loss: 0.7642
Epoch 91/500
65/65 [==============================] - 0s 4ms/step - loss: 0.7680 - val_loss: 0.7641
Epoch 92/500
65/65 [==============================] - 0s 4ms/step - loss: 0.7643 - val_loss: 0.7640
Epoch 93/500


## 2.2 [Detect Anomaly](https://towardsdatascience.com/hands-on-anomaly-detection-with-variational-autoencoders-d4044672acd5)
Detect anomaly by finding those who have a high reconstruction loss.


Steps:
1. Measure error between the original train (clean/normal) set and the output of the model, and generate an error vector representing the error term of each sample.

2. Find a relatively extreme value on that vector to use as your error threshold.

3. Run the model over the test or real data, in which anomalies are probably mixed with normal data.

4. Measure the reconstruction error and mark samples that exhibit an error term higher than the error threshold as anomalies.

In [29]:
def mse(original,reconstructed):
    return np.mean(np.sum((original-reconstructed)**2,axis=1),axis=1)

In [30]:
from sklearn.preprocessing import MinMaxScaler
pred  = vae.predict(df_expand)

VAE_loss = mse(df_expand,pred)
VAE_loss[:20]

array([0.95574188, 0.74010894, 1.40898764, 0.40141323, 0.40382746,
       1.22788972, 4.22813033, 1.03912513, 1.4694415 , 1.82924355,
       0.92050574, 0.37293549, 0.73248267, 1.39929683, 0.94583761,
       1.27355998, 1.23500349, 0.47992888, 0.82621099, 0.74875029])

# 3. VAE + IF

In [32]:
encoder_output = encoder.predict(df_expand)
encoder_output

[array([[ 0.15145175,  0.67342746],
        [ 0.1974955 ,  0.8228035 ],
        [-0.03004303,  1.4391849 ],
        ...,
        [-1.1357707 , -0.25398102],
        [-1.1932938 , -0.20496166],
        [-1.0822445 ,  0.09028078]], dtype=float32),
 array([[-0.6276679 , -0.9370456 ],
        [-0.63363856, -0.9830327 ],
        [-0.7005695 , -1.2157986 ],
        ...,
        [-0.84712607, -0.66307044],
        [-1.0116639 , -0.5885675 ],
        [-1.0018172 , -0.70408064]], dtype=float32),
 array([[ 0.15900487,  0.63194174],
        [ 0.10854987,  0.82873064],
        [-0.04562046,  1.4281056 ],
        ...,
        [-1.2081481 , -0.25612557],
        [-1.2381322 , -0.2533036 ],
        [-1.0661159 ,  0.03189711]], dtype=float32)]

In [33]:
def feature_from_encoder(encoder_output):
    # take mean and log_var as features
    df_encoded_mean   = pd.DataFrame(encoder_output[0],columns=["mean1","mean2"])
    df_encoded_logvar = pd.DataFrame(encoder_output[1],columns=["log_var1","log_var2"])
    df_encoded        = df_encoded_mean.join(df_encoded_logvar)
    return df_encoded

df_encoded = feature_from_encoder(encoder_output)
df_encoded

,mean1,mean2,log_var1,log_var2
0,0.151452,0.673427,-0.627668,-0.937046
1,0.197496,0.822803,-0.633639,-0.983033
2,-0.030043,1.439185,-0.700570,-1.215799
3,0.294835,0.708938,-0.617177,-0.931661
4,0.192892,0.769851,-0.630769,-0.964752
...,...,...,...,...
8274,-1.249233,-0.249652,-0.867745,-0.668523
8275,-1.332658,-0.155256,-0.977586,-0.651610
8276,-1.135771,-0.253981,-0.847126,-0.663070
8277,-1.193294,-0.204962,-1.011664,-0.588567


In [34]:
VAE_IF=IsolationForest(n_estimators=150, 
                       max_samples ='auto', 
                       max_features=1,random_state=42)
VAE_IF.fit(df_encoded)

IsolationForest(max_features=1, n_estimators=150, random_state=42)

In [35]:
VAE_IF_score = -1 * VAE_IF.score_samples(df_encoded)
VAE_IF_score[:20]

array([0.51271279, 0.54316519, 0.57967423, 0.50089449, 0.53539828,
       0.52769579, 0.60597198, 0.48675719, 0.50715018, 0.57402689,
       0.47712399, 0.50684816, 0.4987749 , 0.52490326, 0.5081528 ,
       0.51071541, 0.50904507, 0.53317155, 0.49798758, 0.49262519])

# 4. Combination of results

In [37]:
df['IF_score'] = IF_score
df['IF_score'].describe()

count    8279.000000
mean        0.451213
std         0.030114
min         0.403969
25%         0.428277
50%         0.443766
75%         0.471093
max         0.599004
Name: IF_score, dtype: float64

In [38]:
df['VAE_loss'] = VAE_loss
df['VAE_loss'].describe()

count    8279.000000
mean        0.758763
std         2.107592
min         0.030704
25%         0.209868
50%         0.380608
75%         0.782475
max        98.161029
Name: VAE_loss, dtype: float64

Notice there is some extreme value in `VAE_loss`, which may affect our later visualization 

In [39]:
df['VAE_loss'].quantile(0.99)

5.043369480297673

In [40]:
df['VAE_IF_score'] = VAE_IF_score
df['VAE_IF_score'].describe()

count    8279.000000
mean        0.483081
std         0.041823
min         0.413949
25%         0.450000
50%         0.482546
75%         0.504757
max         0.657840
Name: VAE_IF_score, dtype: float64

In [41]:
df.head()

,DL_TRAFFIC_VOLUME,UL_TRAFFIC_VOLUME,Inter_X2_based_HO_prep,VoLTE_total_traffic,INTRA_FREQ_HO_SR_RATIO,RRC_SR_RATIO,CELL_AVAILABILITY_RATIO,RACH_Stp_Completion_SR_RATIO,Inter_RAT_HO_SR_UTRAN_SRVCC_RATIO,E_RAB_QCI1_DR_RATIO,...,E_UTRAN_Inter_Freq_HO_SR_RATIO,Inter_RAT_HO_SR_GERAN_SRVCC_RATIO,Inter_RAT_Total_HO_SR_RATIO,E_UTRAN_tot_HO_SR_inter_eNB_X2_RATIO,E_RAB_DR_RATIO,HOUR,cell,IF_score,VAE_loss,VAE_IF_score
0,3.779737e+10,3.947172e+09,15.0,4727.0,0.809859,0.992427,1.0,0.962688,0.576414,0.000000,...,0.770642,0.963636,0.963636,0.400000,0.001761,0,0,0.512958,0.955742,0.512713
1,3.684898e+10,4.088752e+09,6.0,3076.0,0.886792,0.993288,1.0,0.973207,0.576414,0.000000,...,0.842105,1.000000,1.000000,0.500000,0.002468,1,0,0.485098,0.740109,0.543165
2,3.292677e+10,5.016897e+09,8.0,3501.0,0.938356,0.994664,1.0,0.966330,0.000000,0.013889,...,0.931624,1.000000,0.966667,0.375000,0.003077,2,0,0.536488,1.408988,0.579674
3,3.021547e+10,5.139107e+09,9.0,2275.0,0.860215,0.994819,1.0,0.943216,0.576414,0.000000,...,0.816901,0.947368,0.947368,0.777778,0.001721,3,0,0.509256,0.401413,0.500894
4,3.082176e+10,4.250716e+09,17.0,2178.0,0.840426,0.995952,1.0,0.936256,0.576414,0.000000,...,0.794521,1.000000,1.000000,0.764706,0.002213,4,0,0.489774,0.403827,0.535398


# 5. Save Results

## 5.1 Save models

In [51]:
#save model
dump(IF,"model/IF.joblib")

['model/IF.joblib']

In [28]:
vae.save("model/VAE")
encoder.save("model/Encoder")
decoder.save("model/Decoder")

INFO:tensorflow:Assets written to: model/VAE\assets
INFO:tensorflow:Assets written to: model/Encoder\assets
INFO:tensorflow:Assets written to: model/Decoder\assets


In [50]:
#save model
dump(VAE_IF,"model/VAE_IF.joblib")

['model/VAE_IF.joblib']

## 5.2 Save Data

In [48]:
df.to_csv("data/KPIs_with_metrics.csv",index=False)